# Importar arquivos

In [4]:
import pickle
with open('data/processed/loan_status.pkl','rb') as f:
    x_treino,y_treino,x_teste,y_teste = pickle.load(f)

# Analise

## Federated Learning

## 1.Install dependencies

In [1]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.7MB 2.8MB/s ta 0:00:011
mkl-random 1.0.1 requires cython, which is not installed.
mkl-fft 1.0.0 requires cython, which is not installed.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [56]:
!pip install -q flwr

In [50]:
!pip install scikit-learn

In [51]:
!pip install --upgrade numpy

## 2.Dividir base de dados

In [19]:

trainloaders = []
valloaders  = []

trainloaders.append(x_treino[0:146,:])
trainloaders.append(x_treino[147:,:])

valloaders.append(y_treino[0:146,])
valloaders.append(y_treino[147:,])



In [16]:
import numpy as np
np.shape(y_treino)

(294,)

## 4.Treinar

In [ ]:
modelo = train(trainloader)
acc = test(dt,x_teste,y_teste)


## 5.Cliente Flower

In [57]:
import flwr as fl

NUM_CLIENTS=2

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, modelo ,trainloader, valloader):
        self.modelo      = modelo  
        self.trainloader = trainloader
        self.valloader   = valloader
    def get_parameters(self, config):
        return 0
  
    def fit(self):
        for x_treino,y_treino in zip(self.trainloader,self.val):
            modelo.fit(x_treino,y_treino)
    
    def evaluate(self):
        
        predicoes = self.modelo.predict(x_teste)
        acc  = accuracy_score(y_teste,predicoes)
        loss = log_loss(y_teste, predicoes)
        
        return loss, len(sel.valloader), {"accuracy": acc}
        

In [45]:
def client_fn(cid):
        model = DecisionTreeClassifier()
        
        trainloader = trainloaders[int(cid)]
        valloader   = valloaders[int(cid)]
        
        return FlowerClient(model,trainloader,valloader)

In [64]:
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        #fraction_evaluate=0.5,
        min_fit_clients=2,
        min_available_clients=2,
)
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy
)

AttributeError: module 'flwr.server' has no attribute 'ServerConfig'